In [5]:
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torch-geometric-temporal

2.0.1+cu118
11.8
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 80.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 20.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.6/107.6 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 53.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch-geometric-temporal: filename=torch_geometric_temporal-0.54.0-py3-none-any.whl size=86722 sha256=66dbca53e08bdaf22077873383a9ade9653c677d3305df62c26ac30fac4d951d
  Stored in directory: /root/.cache/pip/wheels/9e/9b/b6/e15256e053f0cb49b1084a67a709db909d418386a231f0722c
  Created wheel for torch_g

In [7]:
# prompt: mount google drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [61]:
import torch
import os
import json
import numpy as np
from torch_geometric.data import Data

HAND_TO_FACE_THRESHOLD = 0.05

class ASLDatasetLoader:
    def __init__(self, directory_path):
        self.directory_path = directory_path
        self.sign_to_label = self._create_sign_to_label_map()

    def _create_sign_to_label_map(self):
        signs = [os.path.splitext(filename)[0] for filename in os.listdir(self.directory_path)]
        return {sign: i for i, sign in enumerate(signs)}

    def _read_file_data(self, file_path):
        with open(file_path, 'r') as f:
            return json.load(f)

    def _augment_data(self, frame_data, rotation_range=10, translation_range=0.05, scaling_range=0.1):
        """
        Augment the frame data with random rotation, translation, and scaling.

        :param frame_data: Dictionary containing frame landmarks and deltas.
        :param rotation_range: Maximum rotation angle in degrees.
        :param translation_range: Maximum translation as a fraction of landmark range.
        :param scaling_range: Maximum scaling factor.
        :return: Augmented frame data.
        """
        landmarks = np.array(frame_data["landmarks"])
        centroid = np.mean(landmarks, axis=0)

        # Random rotation
        theta = np.radians(np.random.uniform(-rotation_range, rotation_range))
        rotation_matrix = np.array([
            [np.cos(theta), -np.sin(theta)],
            [np.sin(theta), np.cos(theta)]
        ])
        landmarks = np.dot(landmarks - centroid, rotation_matrix) + centroid

        # Random translation
        max_translation = translation_range * (landmarks.max(axis=0) - landmarks.min(axis=0))
        translations = np.random.uniform(-max_translation, max_translation)
        landmarks += translations

        # Random scaling
        scale = np.random.uniform(1 - scaling_range, 1 + scaling_range)
        landmarks = centroid + scale * (landmarks - centroid)

        frame_data["landmarks"] = landmarks.tolist()
        return frame_data

    def _create_graph_from_frame(self, sign_name, frame_data, landmark_types):
        left_hand_indices = [i for i, t in enumerate(landmark_types) if t == "L"]
        right_hand_indices = [i for i, t in enumerate(landmark_types) if t == "R"]
        face_indices = [i for i, t in enumerate(landmark_types) if t == "F"]

        landmarks = np.array(frame_data["landmarks"])
        deltas = np.array(frame_data["deltas"])

        # Create weights based on landmark importance
        weights = [2 if t == "L" or t == "R" else 1 for t in landmark_types]

        # Create edges based on the number of available landmarks (or nodes)
        edges = [[i, i + 1] for i in range(len(landmarks) - 1)]

        # Add edges between the left and right hand landmarks
        for i in left_hand_indices:
            for j in right_hand_indices:
                edges.append([i, j])

        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        # Compute additional features like hand-to-face and hand-to-body distances
        hand_to_face_contact = []
        for idx, ltype in enumerate(landmark_types):
            if ltype in ["L", "R"] and any(t == "F" for t in landmark_types):
                min_distance = min([np.linalg.norm(landmarks[idx] - landmarks[j]) for j, t in enumerate(landmark_types) if t == "F"])
                hand_to_face_contact.append(1 if min_distance < HAND_TO_FACE_THRESHOLD else 0)
            else:
                hand_to_face_contact.append(0)

        # Reshape the 1D arrays to 2D for concatenation
        weights_2d = np.array(weights)[:, np.newaxis]
        hand_to_face_contact_2d = np.array(hand_to_face_contact)[:, np.newaxis]

        # Concatenate landmarks, deltas, importance weights, and hand-to-face contact features
        x = torch.tensor(np.hstack((landmarks, deltas, weights_2d, hand_to_face_contact_2d)), dtype=torch.float)
        y = torch.tensor([self.sign_to_label[sign_name]], dtype=torch.long)

        return Data(x=x, edge_index=edge_index, y=y)


    def get_dataset(self, augment=False):
        dataset = []

        for filename in os.listdir(self.directory_path):
            sign_name = os.path.splitext(filename)[0]
            file_path = os.path.join(self.directory_path, filename)
            sign_data = self._read_file_data(file_path)

            for frame_data in sign_data["frames"]:
                landmark_types = sign_data.get("landmark_types", ["F", "L", "P", "R"])  # defaulting to all types

                if augment:
                  frame_data = self._augment_data(frame_data)
                graph_data = self._create_graph_from_frame(sign_name, frame_data, landmark_types)

                dataset.append(graph_data)

        return dataset

    def number_of_classes(self):
        return len(self.sign_to_label)

In [62]:
from torch_geometric.nn import GCNConv, global_max_pool, BatchNorm  # Notice the change in the import

class GraphClassifier(torch.nn.Module):
    def __init__(self, num_node_features, num_classes, dropout_rate=0.5):
        super(GraphClassifier, self).__init__()
        self.conv1 = GCNConv(num_node_features, 128)
        self.bn1 = BatchNorm(128)
        self.dropout1 = torch.nn.Dropout(dropout_rate)  # Dropout after first layer
        self.conv2 = GCNConv(128, 64)
        self.bn2 = BatchNorm(64)
        self.dropout2 = torch.nn.Dropout(dropout_rate)  # Dropout after second layer
        self.fc = torch.nn.Linear(64, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # First GCN layer
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.leaky_relu(x)  # Use LeakyReLU
        x = self.dropout1(x)

        # Second GCN layer
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.dropout2(x)

        # Global pooling across nodes
        x = global_max_pool(x, data.batch)  # Here's the change from mean pooling to max pooling

        # Final classification layer
        x = self.fc(x)

        return F.log_softmax(x, dim=1)

In [63]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_max_pool, global_mean_pool

class ExtendedGraphClassifier(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(ExtendedGraphClassifier, self).__init__()
        self.conv1 = GCNConv(num_features, 256)  # Increased channels
        self.bn1 = torch.nn.BatchNorm1d(256)    # Batch normalization layer
        self.conv2 = GCNConv(256, 512)          # Increased channels
        self.bn2 = torch.nn.BatchNorm1d(512)    # Batch normalization layer
        self.lin1 = torch.nn.Linear(512, 256)
        self.lin2 = torch.nn.Linear(256, num_classes)
        self.dropout = torch.nn.Dropout(p=0.5)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = self.dropout(x)
        x = global_max_pool(x, batch)
        x = F.relu(self.lin1(x))
        x = self.dropout(x)
        x = self.lin2(x)
        return F.log_softmax(x, dim=1)

In [64]:
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader
from collections import Counter
import random

EPOCHS = 100
LEARNING_RATE = 0.001


def stratified_data_split(data_list, test_size=0.2):
    # Extract labels from data list
    labels = [data.y.item() for data in data_list]

    # Use sklearn's train_test_split with stratify option
    train_data, test_data = train_test_split(data_list, test_size=test_size, stratify=labels, random_state=42)

    return train_data, test_data


def validate(loader, model, device):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        with torch.no_grad():
            out = model(data)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(loader.dataset)

def train():
    directory_path = "/content/drive/MyDrive/Colab Notebooks/DGMD E-14 Project/Datasets/ASL"
    loader = ASLDatasetLoader(directory_path)

    # Create the entire dataset without augmentation and then perform stratified split
    data_list = loader.get_dataset()
    train_dataset, test_dataset = stratified_data_split(data_list, test_size=0.2)

    # Now augment only the training dataset
    augmented_train_dataset = loader.get_dataset(augment=True)

    num_classes = loader.number_of_classes()

    train_labels = [data.y.item() for data in train_dataset]
    test_labels = [data.y.item() for data in test_dataset]

    print("Training label distribution:", Counter(train_labels))
    print("Test label distribution:", Counter(test_labels))

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    num_features = train_dataset[0].x.size(1)
    model = ExtendedGraphClassifier(num_features=num_features, num_classes=num_classes).to(device)

    #model = ExtendedGraphClassifier(num_features=4, num_classes=num_classes).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=5e-4)
    scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.7, patience=5, verbose=True)

    max_epochs_without_improvement = 20
    epochs_without_improvement = 0
    best_val_accuracy = 0

    model.train()
    for epoch in range(EPOCHS):
        total_loss = 0
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()
            out = model(batch)
            loss = F.nll_loss(out, batch.y)
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

            optimizer.step()
            total_loss += loss.item()

            # Check for NaN loss
            if np.isnan(loss.item()):
                print("Warning: NaN loss detected!")

        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch}, Loss: {avg_loss}")

        val_accuracy = validate(test_loader, model, device)
        scheduler.step(val_accuracy)

        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement >= max_epochs_without_improvement:
            print("Early stopping triggered.")
            break

    model.eval()
    correct = 0
    all_preds = []
    all_labels = []

    for batch in test_loader:
        batch = batch.to(device)
        with torch.no_grad():
            pred = model(batch).max(dim=1)[1]
            all_preds.extend(pred.cpu().numpy())
            all_labels.extend(batch.y.cpu().numpy())
            correct += pred.eq(batch.y).sum().item()

    print(f"Accuracy: {correct / len(test_dataset)}")
    print("Sample predictions:", all_preds[:20])
    print("Sample true labels:", all_labels[:20])

In [65]:
train()

Training label distribution: Counter({15: 402, 7: 363, 9: 298, 3: 282, 13: 269, 16: 265, 19: 258, 2: 247, 8: 246, 4: 243, 5: 243, 10: 236, 14: 223, 18: 220, 17: 215, 12: 214, 1: 210, 0: 203, 11: 199, 6: 199})
Test label distribution: Counter({15: 100, 7: 91, 9: 75, 3: 70, 13: 67, 16: 66, 19: 65, 2: 62, 8: 62, 4: 61, 5: 60, 10: 59, 14: 56, 18: 55, 17: 54, 12: 53, 1: 52, 0: 51, 6: 50, 11: 50})
Epoch 0, Loss: 2.7034658178498474
Epoch 1, Loss: 2.227446574953538
Epoch 2, Loss: 1.9220618017112152
Epoch 3, Loss: 1.708973925324935
Epoch 4, Loss: 1.568981236294855
Epoch 5, Loss: 1.4559599579889564
Epoch 6, Loss: 1.3368036660212506
Epoch 7, Loss: 1.3242413526094412
Epoch 8, Loss: 1.2776312152796154
Epoch 9, Loss: 1.2143087224869789
Epoch 10, Loss: 1.175213598375079
Epoch 11, Loss: 1.1082643964622594
Epoch 12, Loss: 1.0993853513198564
Epoch 13, Loss: 1.0614747574812249
Epoch 14, Loss: 1.0249428920730759
Epoch 15, Loss: 0.9787604231623155
Epoch 16, Loss: 0.9762025109574765
Epoch 17, Loss: 0.918809